# Azure AI Agent Service with Bing Grounding

이 노트북은 Azure AI Agent Service와 Bing Grounding Tool을 활용하여, 사용자 질문에 대해 Bing 검색 결과를 기반으로 답변을 생성하는 예제를 다룹니다.

## 주요 내용

- Azure AI Project Client와 Bing Grounding Tool을 초기화합니다.
- Bing 연결 정보를 불러와서 Grounding Tool을 구성합니다.
- 사용자 프롬프트(질문)를 입력하면, Bing 검색을 활용한 답변을 생성하는 Agent를 동적으로 생성하고 실행합니다.
- 각 질문에 대해 에이전트의 답변과 참고한 URL 정보를 출력합니다.

이 과정을 통해, 실제 Bing 검색 결과를 기반으로 한 최신 정보 답변을 Azure AI Agent를 통해 자동화하는 방법을 실습할 수 있습니다.





In [48]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageRole, BingGroundingTool
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import AzureOpenAI
import json
load_dotenv(override=True, dotenv_path="../.env")

credential = DefaultAzureCredential()


In [43]:
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"],
    include_credentials=True
)
conn_id = bing_connection.id

print(conn_id)

bing = BingGroundingTool(connection_id=conn_id)

def populate_answer_agent_with_bing_grounding_tool(prompt: str, project_client: AIProjectClient, bing):
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
        top_p=0.9,
    )

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
        
    )

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, max_completion_tokens=1500)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    project_client.agents.delete_agent(agent.id)
    response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
        MessageRole.AGENT
    )

    return  response_message



def display_markdown_content(content: str):
    display(Markdown(content))

/subscriptions/ef750ece-3939-4a4b-ade4-cae9eabcf312/resourceGroups/rg-writer/providers/Microsoft.MachineLearningServices/workspaces/ai-project-q4u67epqs2xo4/connections/agentbingsearch


In [54]:
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.
            
            1. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘            
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]



for input in inputs:
    print(f"User input: {input}")
    rewritten_query = rewrite_query_for_search_and_llm(input, client)
    response_message = populate_answer_agent_with_bing_grounding_tool(rewritten_query['llm_query'], project_client, bing)
    if response_message:
        for text_message in response_message.text_messages:
            display_markdown_content(text_message.text.value)
        # for annotation in response_message.url_citation_annotations:
        #     print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")


User input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


삼성전자 인덕션 제품 중 3구 이상 또는 다양한 화구 수를 가진 모델들을 추천해 드리겠습니다. 각 모델의 주요 기능과 장점도 함께 안내해 드립니다.

---

## 1. 삼성 BESPOKE 인덕션 3구 모델

### 주요 기능
- **3구 인덕션 화구 구성**으로 한 번에 다양한 요리 가능
- **스마트 화구 제어**: 각 화구를 독립적으로 제어해 최적의 조리 환경 제공
- **와이드 인덕션 화구**: 큰 냄비나 팬도 문제없이 사용 가능
- **자동 안전 기능**: 과열, 잔열 표시, 잠금 기능 등 안전장치 탑재
- **심플하고 세련된 디자인**으로 주방 인테리어와 잘 어울림

### 장점
- 여러 요리를 동시에 할 수 있어 시간 절약
- 강력한 화력과 정밀한 온도 조절 가능
- 사용자 맞춤형 조리 설정 가능

---

## 2. 삼성 BESPOKE 인덕션 4구 모델

### 주요 기능
- **4개의 독립 화구**로 대가족 또는 많은 요리를 한꺼번에 할 때 적합
- **전기 화구와 인덕션 화구 혼합 가능 모델(선택 모델에 따라 다름)**
- **터치 방식의 직관적 컨트롤 패널**
- **와이드 인덕션 존**으로 대형 용기 조리 가능
- **인공지능 기반 자동 조리 프로그램** 내장

### 장점
- 다양한 크기와 형태의 조리 용기에 대응 가능
- 고성능 인덕션 화력으로 빠르고 균일한 조리
- 안전성과 편리성 극대화

---

## 3. 삼성 BESPOKE 인덕션 5구 이상 모델 (일부 프리미엄 라인)

### 주요 기능
- 5개 이상의 다중 화구 구성으로 전문 요리 환경 조성
- 각 화구 별 세밀한 온도 및 시간 제어 기능
- **Wi-Fi 연동 스마트홈 호환 가능**: 스마트폰 앱으로 원격 조작 가능
- **잔열 표시 및 자동 전원 차단 기능** 내장
- 넓은 조리 공간과 세련된 디자인

### 장점
- 대규모 요리 및 다양한 조리 작업에 최적
- 편리한 스마트 기능으로 사용 편의성 극대화
- 고급 주방 인테리어에 어울리는 디자인

---

원하는 화구 수에 따라 3구, 4구, 5구 이상 모델을 선택할 수 있으며, 모두 강력한 인덕션 화력과 안전 기능, 세련된 디자인이 특징입니다. 구체적인 모델명과 최신 출시 정보는 삼성전자 공식 홈페이지 또는 가까운 매장에서 확인하시면 좋습니다.

필요하시면 특정 모델별 상세 스펙도 추가로 안내해 드릴 수 있습니다.

User input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


부모님께 선물하기 좋은 삼성전자 TV 모델을 추천드리자면, 다음과 같은 기준을 고려했습니다.

- **화면 크기:** 거실 환경과 시청 거리 고려, 50~65인치가 무난
- **화질:** 선명하고 자연스러운 화질, HDR 지원
- **기능:** 스마트 TV 기능, 음성 인식, 간편한 UI, 시니어 친화적 기능 포함

---

## 추천 모델: 삼성전자 **The Frame 55형 (2024년형) QLED TV**

### 주요 특징

1. **화면 크기 & 디자인**
   - 55인치 화면으로 거실에서 보기 적당한 크기
   - 액자형 디자인으로 평소에는 예술 작품처럼 벽에 걸어 둘 수 있음
   - 세련된 디자인이 인테리어 효과도 뛰어남

2. **화질**
   - QLED 디스플레이로 밝고 선명한 화질 제공
   - 4K 해상도, HDR10+ 지원으로 영화, 드라마, 스포츠 등 다양한 콘텐츠를 선명하게 감상 가능
   - AI 업스케일링 기능으로 FHD 영상도 4K 수준으로 선명하게 변환

3. **사용 편의성**
   - 스마트 허브와 Tizen OS 탑재로 다양한 OTT 서비스(넷플릭스, 유튜브 등) 간편하게 이용 가능
   - 음성 인식 리모컨 지원으로 리모컨 버튼을 일일이 누르지 않고 음성으로 조작 가능
   - 눈부심 방지 패널로 장시간 시청에도 눈의 피로 최소화

4. **시니어 맞춤 기능**
   - 메뉴가 직관적이고 크게 표시되어 쉽게 조작 가능
   - 자주 쓰는 앱 바로가기 기능으로 편리성 증가
   - 주변 밝기에 따라 화면 밝기 자동 조절

---

### 요약
| 항목        | 내용                          |
|-------------|-----------------------------|
| 모델명      | 삼성 The Frame 55형 (2024)   |
| 화면 크기   | 55인치                       |
| 해상도      | 4K QLED                      |
| 주요 기능   | AI 업스케일링, HDR10+, 음성 리모컨, 스마트 허브 |
| 디자인     | 액자형, 벽걸이 가능           |
| 시니어 적합 | 큰 글자 메뉴, 음성 인식, 눈부심 방지 |

---

이 모델은 기능과 디자인, 사용 편의성 모두 균형 잡혀 있어 부모님께 선물하기 매우 적합합니다. 특히, 평소 TV가 인테리어 소품 역할도 하길 원하시면 The Frame 시리즈가 훌륭한 선택입니다.

필요하시면 설치 방법이나 사용법도 상세히 안내해 드릴 수 있습니다!

User input: 삼성전자 25년 제품이 작년 대비 좋아진것은


...

삼성전자에서 판매하는 오디오 브랜드인 JBL과 하만카돈(Harman Kardon)은 모두 하만(Harman) 산하의 브랜드로, 각기 다른 음향 철학과 제품 특성을 가지고 있습니다. 주요 차이점을 음향 기술, 음질 특성, 제품 라인업, 그리고 타깃 소비자층 측면에서 설명드리겠습니다.

---

## 1. 음향 기술

- **JBL**
  - 강력한 베이스와 높은 출력에 중점을 둔 사운드 튜닝.
  - 스포츠, 아웃도어 환경에 적합한 내구성과 방수 기술 적용 (예: IPX 등급).
  - 파워풀한 음향을 위해 고성능 우퍼와 트위터를 조합하는 경향.
  
- **하만카돈 (Harman Kardon)**
  - 섬세하고 균형 잡힌 사운드, 특히 중음과 고음에 강점.
  - 고급 오디오 기기에서 사용되는 프리미엄 DSP(디지털 신호 처리) 기술 적용.
  - 인테리어와 잘 어우러지는 디자인과 함께 최적화된 음향 공간 설계.

---

## 2. 음질 특성

- **JBL**
  - 다이나믹하고 강렬한 사운드, 특히 저음역대가 강조됨.
  - 콘서트나 클럽 같은 환경에 어울리는 '파워풀한' 음향.
  - 선명한 고음과 명확한 중음으로 활동적인 분위기 연출.

- **하만카돈**
  - 부드럽고 자연스러운 사운드, 고급 오디오를 지향하는 음색.
  - 음향의 디테일과 깊이감이 뛰어나며, 음악 감상에 최적화됨.
  - 고음과 중음의 밸런스가 잘 맞아 클래식, 재즈 등 섬세한 음악에 적합.

---

## 3. 제품 라인업

- **JBL**
  - 포터블 블루투스 스피커, 헤드폰, 사운드바, 자동차용 오디오 등 광범위.
  - 다양한 크기와 가격대 제품으로 캐주얼 사용자 대상.
  - 액티브 라이프스타일을 겨냥한 튼튼하고 실용적인 디자인.

- **하만카돈**
  - 하이엔드 스피커, 사운드바, 헤드폰, 홈 시어터 시스템 등 프리미엄 제품 위주.
  - 세련된 디자인과 고급 소재 사용.
  - 인테리어와 조화를 이루는 고급 오디오 솔루션 집중.

---

## 4. 타깃 소비자층

- **JBL**
  - 젊은 층, 야외 활동을 즐기는 사용자, 활동적이고 역동적인 라이프스타일을 가진 소비자.
  - 가격 대비 성능을 중시하는 일반 사용자.
  - 음악을 큰 볼륨으로 즐기며 휴대성을 중시하는 소비자.

- **하만카돈**
  - 고품질 오디오를 선호하는 오디오 애호가, 하이엔드 시장 타깃.
  - 실내 인테리어와 조화를 이루는 디자인과 음질을 원하는 사용자.
  - 홈 시어터 및 고급 음향 시스템 구축을 원하는 고객.

---

요약하면, **JBL은 강력한 베이스와 휴대성, 실용성을 중시하는 대중적이고 활동적인 오디오 브랜드**이며, **하만카돈은 고급스럽고 정교한 음질, 세련된 디자인으로 하이엔드 오디오 시장을 겨냥**한 브랜드입니다.

필요하시면 삼성전자 공식 홈페이지나 하만 산하 브랜드별 세부 제품 정보도 안내해 드릴 수 있습니다.

User input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요


갤럭시 버즈 이어버드 한쪽만 새로 구매하여 기존 이어버드와 함께 사용하려면, 새 이어버드와 기존 이어버드를 올바르게 페어링(재연결)하는 과정이 필요합니다. 아래는 단계별 방법과 주의사항입니다.

---

## 1. 준비 사항
- 기존 이어버드와 새로 구매한 한쪽 이어버드를 모두 준비합니다.
- 갤럭시 버즈 충전 케이스가 완전히 충전되어 있어야 합니다.
- 스마트폰(갤럭시 또는 안드로이드/아이폰 모두 가능)과 블루투스가 켜져 있어야 합니다.

---

## 2. 한쪽 이어버드만 새로 교체했을 때 페어링 과정

### 1) 새 이어버드를 충전 케이스에 넣기
- 새로 구매한 이어버드를 충전 케이스에 넣고 뚜껑을 닫습니다.
- 기존 이어버드도 충전 케이스에 함께 넣어둡니다.
- 이 상태에서 약 7~10초간 기다려 두 이어버드가 케이스와 동기화되도록 합니다.

### 2) 이어버드 초기화(재설정)
- 이어버드가 제대로 페어링되지 않는 경우, 초기화를 진행합니다.
- 초기화 방법:
  - 이어버드와 충전 케이스 뚜껑을 열고 이어버드를 모두 꺼냅니다.
  - 이어버드를 다시 충전 케이스에 넣은 후, 뚜껑을 닫았다가 다시 엽니다.
  - 갤럭시 웨어러블 앱(Galaxy Wearable App)에서 이어버드 초기화 기능을 사용하거나,
  - 이어버드를 10초 이상 눌러 초기화합니다(모델별 차이 있음).

### 3) 스마트폰과 페어링
- 스마트폰 블루투스 설정에서 기존 갤럭시 버즈 연결을 삭제(잊기)합니다.
- 갤럭시 웨어러블 앱을 실행하고, ‘새 기기 연결’ 또는 ‘이어버드 다시 연결’을 선택합니다.
- 이어버드를 충전 케이스에서 꺼내면 페어링 모드로 진입합니다.
- 스마트폰에서 ‘Galaxy Buds’ 이름을 찾아 선택해 연결합니다.

### 4) 페어링 확인
- 페어링 완료 후, 갤럭시 웨어러블 앱에서 양쪽 이어버드 상태(배터리 잔량 등)를 확인하세요.
- 새로 교체한 한쪽 이어버드가 제대로 인식되는지 확인합니다.

---

## 3. 주의사항
- **한쪽 이어버드만 교체한 경우, 반드시 기존 이어버드와 새 이어버드가 동일한 모델(예: 갤럭시 버즈+ 좌우)인지 확인하세요.** 서로 다른 모델은 페어링이 불가할 수 있습니다.
- 새 이어버드가 기존 이어버드와 완전히 동기화되지 않으면 사운드 불균형, 연결 끊김 등의 문제가 발생할 수 있습니다.
- 이어버드와 충전 케이스를 완전히 충전한 상태에서 작업하세요.
- 초기화 후에는 이어버드의 모든 사용자 설정이 초기화될 수 있습니다.
- 페어링 문제 발생 시, 갤럭시 웨어러블 앱을 최신 버전으로 업데이트하거나 스마트폰을 재부팅 후 다시 시도해보세요.

---

필요시 모델명(예: 갤럭시 버즈, 버즈+, 버즈 프로 등)을 알려주시면 더 구체적인 안내도 가능합니다.

User input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나


삼성전자 모니터 S25 모델과 S24 모델의 무게 차이에 대해 아래와 같이 정리했습니다.

- S24 모델 무게: 약 167g
- S25 모델 무게: 약 162g

즉, S25 모델이 S24 모델보다 약 5g 가볍습니다. 

이 차이는 S25가 소재 개선과 내부 부품 최적화를 통해 더 얇고 가벼운 구조로 제작된 결과입니다. 무게 차이는 크지 않지만, 경량화된 점이 사용자 경험에 긍정적인 영향을 줄 수 있습니다.

또한 S25는 두께도 S24보다 약간 얇아져 휴대성과 그립감이 더 좋아졌습니다.

요약:
| 모델 | 무게  | 두께    |
|-------|--------|---------|
| S24   | 167g   | 7.6mm   |
| S25   | 162g   | 7.2mm   |

따라서 S25는 S24 대비 5g 가벼워졌고, 두께도 0.4mm 얇아진 점이 특징입니다.

필요하신 경우 더 자세한 스펙 정보도 안내해 드릴 수 있습니다.  

출처: 삼성 갤럭시 S25, S24 스펙 비교 자료 및 리뷰 글 종합 정리 【9:6†source】【9:0†source】